Resources:
- https://docs.python.org/3/tutorial/datastructures.html#dictionaries
- https://apple.github.io/turicreate/docs/api/generated/turicreate.SArray.apply.html?highlight=apply#turicreate.SArray.apply
- https://scikit-learn.org/stable/

# Analyze Product Sentiment

In [1]:
import turicreate
import pandas as pd

# Read product review data

In [2]:
# products = turicreate.SFrame('../input/amazon_baby.sframe')

# Explore data

In [3]:
# products.head(3)

# Task1
Use .apply() to build a new feature with the counts for each of the selected_words: In the notebook above, we created a column ‘word_count’ with the word counts for each review. Our first task is to create a new column in the products SFrame with the counts for each selected_word above, and, in the process, we will see how the method .apply() can be used to create new columns in our data (our features) and how to use a Python function, which is an extremely useful concept to grasp!

In [4]:
# with turicreate is way easier
# products.groupby('name',operations={'count':turicreate.aggregate.COUNT()}).sort('count',ascending=False)

In [5]:
# build the column word_count
# products['word_count'] = graphlab.text_analytics.count_words(products['review'])

In [6]:
products = pd.read_csv('../input/amazon_baby.csv')
products.head()

,name,review,rating
0,Planetwise Flannel Wipes,"These flannel wipes are OK, but in my opinion ...",3
1,Planetwise Wipe Pouch,it came early and was not disappointed. i love...,5
2,Annas Dream Full Quilt with 2 Shams,Very soft and comfortable and warmer than it l...,5
3,Stop Pacifier Sucking without tears with Thumb...,This is a product well worth the purchase. I ...,5
4,Stop Pacifier Sucking without tears with Thumb...,All of my kids have cried non-stop when I trie...,5


In [7]:
selected_words = ['awesome', 'great', 'fantastic', 'amazing', 'love', 'horrible', 'bad', 'terrible', 'awful', 'wow', 'hate']

In [8]:
 # fill in N/A's in the review column. Important step
products = products.fillna({'review':''})

In [9]:
for word in selected_words:
    products[word] = products['review'].apply(lambda text: text.split().count(word))

In [10]:
products.head()

,name,review,rating,awesome,great,fantastic,amazing,love,horrible,bad,terrible,awful,wow,hate
0,Planetwise Flannel Wipes,"These flannel wipes are OK, but in my opinion ...",3,0,0,0,0,0,0,0,0,0,0,0
1,Planetwise Wipe Pouch,it came early and was not disappointed. i love...,5,0,0,0,0,1,0,0,0,0,0,0
2,Annas Dream Full Quilt with 2 Shams,Very soft and comfortable and warmer than it l...,5,0,0,0,0,0,0,0,0,0,0,0
3,Stop Pacifier Sucking without tears with Thumb...,This is a product well worth the purchase. I ...,5,0,0,0,0,2,0,0,0,0,0,0
4,Stop Pacifier Sucking without tears with Thumb...,All of my kids have cried non-stop when I trie...,5,0,1,0,0,0,0,0,0,0,0,0


### question 1 and 2

In [11]:
sorted_words=[]
for w in selected_words:
    print(w,products[w].sum())

('awesome', 1683)
('great', 37056)
('fantastic', 807)
('amazing', 1164)
('love', 33667)
('horrible', 637)
('bad', 3599)
('terrible', 659)
('awful', 337)
('wow', 54)
('hate', 1089)


# Examining reviews of the jiraffe toy

In [12]:
giraffe_reviews = products[products['name'] == 'Vulli Sophie the Giraffe Teether']
giraffe_reviews.head()

,name,review,rating,awesome,great,fantastic,amazing,love,horrible,bad,terrible,awful,wow,hate
34313,Vulli Sophie the Giraffe Teether,He likes chewing on all the parts especially t...,5,0,0,0,0,0,0,0,0,0,0,0
34314,Vulli Sophie the Giraffe Teether,My son loves this toy and fits great in the di...,5,0,1,0,0,0,0,0,0,0,0,0
34315,Vulli Sophie the Giraffe Teether,There really should be a large warning on the ...,1,0,0,0,0,0,0,0,0,0,0,0
34316,Vulli Sophie the Giraffe Teether,All the moms in my moms\' group got Sophie for...,5,0,0,0,0,1,0,0,0,0,0,0
34317,Vulli Sophie the Giraffe Teether,I was a little skeptical on whether Sophie was...,5,0,0,0,0,0,0,0,0,0,0,0


# Define what is positive and negative sentiment

In [13]:
#ignore all 3*  reviews. We will ignore all reviews with rating = 3, since they tend to have a neutral sentiment. 
# Reviews with a rating of 4 or higher will be considered positive, while the ones with rating of 2 or lower will 
# have a negative sentiment.

products = products[products['rating']!= 3]

In [14]:
products.rating.describe()

count    166752.000000
mean          4.233191
std           1.295527
min           1.000000
25%           4.000000
50%           5.000000
75%           5.000000
max           5.000000
Name: rating, dtype: float64

In [17]:
#positive sentiment = 4* or 5* reviews. with this we'll make a binary classification
products['sentiment'] = products['rating'].apply(lambda rating : +1 if rating > 3 else -1)

In [20]:
# !! 
products.sentiment.value_counts()

 1    140259
-1     26493
Name: sentiment, dtype: int64

In [ ]:
# highly skeweed

# Train our sentiment classifier

In [21]:
import numpy as np
from sklearn.model_selection import train_test_split
train_data , test_data = train_test_split(products,test_size=0.2,random_state=42) #this function already  merges the dataset before spitting

In [22]:
train_data.shape

(133401, 15)

In [23]:
test_data.shape

(33351, 15)

In [24]:
#Step 1. Import the model I want to use
from sklearn.linear_model import LogisticRegression

#Step 2. Make an instance of the Model
logisticRegr = LogisticRegression()
#logisticRegr2 = linear_model.LogisticRegression()

#Step 3. Training the model on the data, storing the information learned from the data
selected_words_model = logisticRegr.fit(train_data[selected_words], train_data['sentiment'])

### 3. Out of the 11 words in selected_words, which one got the most positive weight in the selected_words_model?

### 4. Out of the 11 words in selected_words, which one got the most negative weight in the selected_words_model?

In [26]:
selected_words_model.coef_[0]

array([ 0.8154788 ,  0.72506839,  0.73230086,  0.80352209,  1.34444403,
       -1.71696588, -0.95817482, -1.95267992, -1.61337755, -0.10502929,
       -1.43882078])

In [29]:
feature_weight = pd.DataFrame(
    {'feature': selected_words,
     'weight': selected_words_model.coef_[0]
    })
feature_weight.sort_values(by=["weight"],inplace=True, ascending=False)
feature_weight

,feature,weight
4,love,1.344444
0,awesome,0.815479
3,amazing,0.803522
2,fantastic,0.732301
1,great,0.725068
9,wow,-0.105029
6,bad,-0.958175
10,hate,-1.438821
8,awful,-1.613378
5,horrible,-1.716966


### 5. Which of the following ranges contains the accuracy of the selected_words_model on the test_data?

In [33]:
from sklearn.metrics import accuracy_score
y_pred_selected_words = selected_words_model.predict(test_data[selected_words])
y_true_selected_words = test_data['sentiment']
accuracy_score(y_true_selected_words, y_pred_selected_words)
print('accuracy of selected words model is',accuracy_score(y_true_selected_words, y_pred_selected_words))

('accuracy of selected words model is', 0.8443824772870379)


In [31]:
# imbalanced dataset... accuracy is not a good metric

### 6. Which of the following ranges contains the accuracy of the sentiment_model in the IPython Notebook from lecture on the test_data?

In [36]:
# We will now compute the word count for each word that appears in the reviews.
from sklearn.feature_extraction.text import CountVectorizer

vectorizer = CountVectorizer(token_pattern=r'\b\w+\b')
     # Use this token pattern to keep single-letter words
# First, learn vocabulary from the training data and assign columns to words
# Then convert the training data into a sparse matrix
train_matrix = vectorizer.fit_transform(train_data['review'])

# Second, convert the test data into a sparse matrix, using the same word-column mapping
test_matrix = vectorizer.transform(test_data['review'])

In [37]:
train_matrix.shape

(133401, 57522)

In [38]:
test_matrix.shape

(33351, 57522)

In [39]:
# Training the model on the data, storing the information learned from the data
sentiment_model = LogisticRegression().fit(train_matrix, train_data['sentiment'])

In [40]:
y_pred_sentiment_model = sentiment_model.predict(test_matrix)
y_true_sentiment_model = test_data['sentiment']
accuracy_score(y_pred_sentiment_model, y_true_sentiment_model)
print('accuracy of sentiment model is',accuracy_score(y_pred_sentiment_model, y_true_sentiment_model))

('answer: the accuracy of sentiment model is', 0.9351743575904771)


### 7. Which of the following ranges contains the accuracy of the majority class classifier, which simply predicts the majority class on the test_data?

In [60]:
# the majority of train data
num_positive_train  = (train_data['sentiment'] == +1).sum()
num_negative_train = (train_data['sentiment'] == -1).sum()
print ("positive_train: {}".format(num_positive_train), 
       "negative_train: {}".format(num_negative_train))


('positive_train: 112182', 'negative_train: 21219')


In [61]:
num_positive_test  = (test_data['sentiment'] == +1).sum()
num_negative_test = (test_data['sentiment'] == -1).sum()
print(num_positive_test, num_negative_test)

(28077, 5274)


In [62]:
print(num_positive_test/(num_positive_test+num_negative_test))
#wtf

### 8. How do you compare the different learned models with the baseline approach where we are just predicting the majority class?

### 9. Which of the following ranges contains the ‘predicted_sentiment’ for the most positive review for ‘Baby Trend Diaper Champ’, according to the sentiment_model from the IPython Notebook from lecture?

In [63]:
diaper_champ_reviews = test_data[test_data['name'] == 'Baby Trend Diaper Champ']
diaper_champ_reviews['review']

382    I changed my registry from the Diaper Genie to...
387    So far so good.  I was using the cheapie pail....
528    As a Mother of 9 month old twins, I go through...
353    In reading some of the other reviews, I think ...
455    We put a Diaper Genie on the shower list for t...
404    I love the diaper champ.  I registered for thi...
571    We did alot of research on diaper pails before...
643    I\'ve read all of the reviews of those of you ...
512    Bottom line: Using regular bags saves big time...
541    This was great for first 5 months of my baby g...
325    We bought this pail because it used regular tr...
523    We researched all of the different types of di...
458    I\'m SO glad that we asked the sales associate...
428    I got this for my daughter and I just think it...
572    We registered for the diaper champ after compa...
415    This is a wonderful product.  One suggestion t...
548    This is one of the best things that we got as ...
347    I\'ve been using this di

In [64]:
# use the sentiment_model to predict the sentiment of each review in diaper_champ_reviews and sort the results 
# according to their ‘predicted_sentiment’.


In [66]:
test_matrix_diaper_champ = vectorizer.transform(diaper_champ_reviews['review'])
test_matrix_diaper_champ

<64x57522 sparse matrix of type '<type 'numpy.int64'>'
	with 5080 stored elements in Compressed Sparse Row format>

In [67]:
diaper_sentiment_model_pred_proba = sentiment_model.predict_proba(test_matrix_diaper_champ)
diaper_sentiment_model_pred = sentiment_model.predict(test_matrix_diaper_champ)
len(diaper_sentiment_model_pred_proba[:,1])

64

In [68]:
diaper_review_sentiment_pred_proba = pd.DataFrame(
    {'review_clean': diaper_champ_reviews['review'],
     'predicted_sentiment_prob': diaper_sentiment_model_pred_proba[:,1].tolist(),
     'predicted_sentiment': diaper_sentiment_model_pred.tolist()
    })

In [69]:
diaper_review_sentiment_pred_proba.sort_values(by='predicted_sentiment_prob', ascending=False)

,predicted_sentiment,predicted_sentiment_prob,review_clean
571,1,0.999999,We did alot of research on diaper pails before...
330,1,0.999993,Although I had a Diaper Genie from my first ch...
635,1,0.999987,my son is now 3 months old and when he was bor...
548,1,0.999985,This is one of the best things that we got as ...
428,1,0.999963,I got this for my daughter and I just think it...
458,1,0.999962,I\'m SO glad that we asked the sales associate...
509,1,0.999959,My husband and I wanted this style (vs. the Di...
512,1,0.999936,Bottom line: Using regular bags saves big time...
528,1,0.999933,"As a Mother of 9 month old twins, I go through..."
329,1,0.999922,I LOVE LOVE LOVE this product! It is SO much e...


### 10. Consider the most positive review for ‘Baby Trend Diaper Champ’ according to the sentiment_model from the IPython Notebook from lecture. Which of the following ranges contains the predicted_sentiment for this review, if we use the selected_words_model to analyze it?